In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import gdown
from thefuzz import process

In [32]:
#Downloading data
file_id_1 = '1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi'
file_id_2 = '1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG'
file_id_3 = '1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2'
url_1 = f"https://drive.google.com/uc?id={file_id_1}"
url_2 = f"https://drive.google.com/uc?id={file_id_2}"
url_3 = f"https://drive.google.com/uc?id={file_id_3}"
output_1 = 'data_dv.csv'
output_2 = 'data_ev.csv'
output_3 = 'data_ur.csv'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)
gdown.download(url_3, output_3, quiet=False)
df1 = pd.read_csv(output_1)
df2 = pd.read_csv(output_2)
df3 = pd.read_csv(output_3,sep = ';')

# Viewing data
df1.head()
df2.head()
df3.head()

#Cleaning data and changing name col
df1['disitrict_name'] = df1['disitrict_name'].astype(str)
df2['Disitrict'] = df2['Disitrict'].astype(str)
df2['district_clean'] = df2['Disitrict'].apply(lambda x:process.extractOne(x,df1['disitrict_name'])[0])
df3.drop(columns=['Kod','Unnamed: 3'], inplace=True)
df3.rename(columns={'Nazwa':'district_name','ogółem;2023;[%]':'unemployment_rate'},inplace=True)
df3['district_clean3'] = df3['district_name'].apply(lambda x: process.extractOne(x,df1['disitrict_name'])[0])

#Merging data into one DF
df_merged = df1.merge(df2,left_on='disitrict_name',right_on='district_clean',how='inner')
df_merged = df_merged.merge(df3,left_on='disitrict_name',right_on='district_clean3', how='inner')
df_merged.head()
df_merged.rename(columns = {'disitrict_name':'district_name'},inplace=True)
df_merged.drop(columns=['Disitrict','district_clean','district_clean3'], inplace=True)
df_merged.dropna(inplace=True)
df_merged.info()
float_list = ['salary','education_expenses']
for i in float_list:
    df_merged[i] = df_merged[i].astype(str)
    df_merged[i] = df_merged[i].apply(lambda x:x.replace(',',''))
    df_merged[i] = df_merged[i].astype(float)

df_merged['unemployment_rate'] = df_merged['unemployment_rate'].str.replace(',','.')
df_merged['unemployment_rate'] = df_merged['unemployment_rate'].astype(float)
df_merged.drop_duplicates(subset = ['district_name'], keep ='first', inplace = True)
df_merged.head()


Downloading...
From: https://drive.google.com/uc?id=1s0zEHlvgbIr4AKruko2yFQymK_Pp1QNi
To: C:\Users\dawid\OneDrive\Data_Science\DAM\DAM_poster\data_dv.csv
100%|██████████| 9.14k/9.14k [00:00<00:00, 1.56MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sTnKn_XSgZ_8uzNo8INDtmz9WjIqziYG
To: C:\Users\dawid\OneDrive\Data_Science\DAM\DAM_poster\data_ev.csv
100%|██████████| 17.4k/17.4k [00:00<00:00, 2.31MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WuCNd7d4ZUJ8OQPmGtvMk_2Fasr9v0v2
To: C:\Users\dawid\OneDrive\Data_Science\DAM\DAM_poster\data_ur.csv
100%|██████████| 13.3k/13.3k [00:00<00:00, 2.67MB/s]


<class 'pandas.core.frame.DataFrame'>
Index: 514 entries, 0 to 612
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   district_name       514 non-null    object 
 1   mean_score          514 non-null    float64
 2   salary              514 non-null    object 
 3   education_expenses  514 non-null    object 
 4   year                514 non-null    float64
 5   divorces            514 non-null    float64
 6   district_name       514 non-null    object 
 7   unemployment_rate   514 non-null    object 
dtypes: float64(3), object(5)
memory usage: 36.1+ KB


,district_name,mean_score,salary,education_expenses,year,divorces,district_name,unemployment_rate
0,aleksandrowski,71.580247,6019.18,857.55,2023.0,11.6,Powiat aleksandrowski,10.9
1,brodnicki,74.575000,5919.35,155.69,2023.0,11.9,Powiat brodnicki,6.7
2,bydgoski,47.770270,6461.57,635.18,2023.0,19.4,Powiat bydgoski,3.1
3,Bydgoszcz,72.275621,7478.19,566.55,2023.0,19.9,Powiat m. Bydgoszcz,2.2
4,golubsko-dobrzy?ski,78.767857,6678.54,696.36,2023.0,10.1,Powiat golubsko-dobrzyński,10.2


In [33]:
#Linear regression OLS method
#df_merged.info()
Y = df_merged['mean_score']
ev_OLS = ['salary','education_expenses','divorces','unemployment_rate']
X = df_merged[ev_OLS]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             mean_score   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     6.306
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           6.92e-05
Time:                        12:40:51   Log-Likelihood:                -1055.6
No. Observations:                 309   AIC:                             2121.
Df Residuals:                     304   BIC:                             2140.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 61.7610      4